# Code to load in Indego/Weather data to create Pickle Files

In [2]:
# Import all necessary libraries to load in and manipulate data to create pickled files that
# can continue to be used later, so that Darksky API doesn't need to be called multiple times

from darksky import forecast
from datetime import datetime as dt 
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import os
import pickle 

In [2]:
# Create variable for Darksky API using environment variable to keep the key private 

API_Key = os.environ.get('DSKEY')

In [3]:
# Collect all Indgo files in a list to be read in 

directory = './Downloads'         
pattern = 'trips' 
matching_files = [f for f in os.listdir(directory) if pattern in f.casefold() and f.endswith(".csv")]

In [4]:
# Read in all Indego data by feeding filenames into read_csv and concat them into one dataframe

df_list = []
for i in matching_files:
    df_temp = pd.read_csv('./Downloads/' + i , low_memory = False)
    df_list.append(df_temp)
    
df = pd.concat(df_list, sort=True)

In [5]:
# Call info to examine datatypes and memory usage as well as check for possible missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3524699 entries, 0 to 119558
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   bike_id              object 
 1   bike_type            object 
 2   duration             int64  
 3   end_lat              object 
 4   end_lon              object 
 5   end_station          float64
 6   end_station_id       float64
 7   end_time             object 
 8   passholder_type      object 
 9   plan_duration        float64
 10  start_lat            object 
 11  start_lon            object 
 12  start_station        float64
 13  start_station_id     float64
 14  start_time           object 
 15  trip_id              int64  
 16  trip_route_category  object 
dtypes: float64(5), int64(2), object(10)
memory usage: 484.0+ MB


In [6]:
# Through data exploration unwanted string values had been discovered and need to be removed 

df.drop(labels = df[df['start_lat'] == '\\N'].index, inplace= True)
df.drop(labels = df[df['start_lon'] == '\\N'].index, inplace= True)
df.drop(labels = df[df['end_lat'] == '\\N'].index, inplace= True)
df.drop(labels = df[df['end_lon'] == '\\N'].index, inplace= True)
df.drop(labels = df[df['bike_id'] == 'delete me'].index, inplace= True)

In [7]:
# Adjust datatypes in order to reduce memory footprint where possible 

df = df.astype({'start_lat' : 'float16' , 'end_lat' : 'float16' , 'start_lon' : 'float16' , 'end_lon' : 'float16',
                'bike_type' : 'category' , 'passholder_type' : 'category' , 'plan_duration' : 'float16',
                'end_station' : 'float32' , 'end_station_id' : 'float32' , 'start_station' : 'float32',
                'start_station_id' : 'float32' , 'trip_id' : 'int32' , 'trip_route_category' : 'category'})

In [8]:
# Cast start and end time variables to date time objects to handle needed operations later 

df['start_time'] = pd.to_datetime(df['start_time'], infer_datetime_format=True)
df['end_time'] = pd.to_datetime(df['end_time'], infer_datetime_format=True)

In [9]:
# Create new columns for date, year, month, and day of week to use for later analysis 

df['Date'] = df['start_time'].dt.date
df['Year'] = df['start_time'].dt.year
df['Month'] = df['start_time'].dt.month
df['Day'] = df['start_time'].dt.weekday

In [10]:
# Cast our new columns to continue minimizing our dataframes memory footprint 

df = df.astype({'Year' : 'int16' , 'Month' : 'int8' , 'Day' : 'int8'})

In [11]:
# Check info again to see that all of our casting worked correctly and our memory usage has been lowered 

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3503654 entries, 0 to 119558
Data columns (total 21 columns):
 #   Column               Dtype         
---  ------               -----         
 0   bike_id              object        
 1   bike_type            category      
 2   duration             int64         
 3   end_lat              float16       
 4   end_lon              float16       
 5   end_station          float32       
 6   end_station_id       float32       
 7   end_time             datetime64[ns]
 8   passholder_type      category      
 9   plan_duration        float16       
 10  start_lat            float16       
 11  start_lon            float16       
 12  start_station        float32       
 13  start_station_id     float32       
 14  start_time           datetime64[ns]
 15  trip_id              int32         
 16  trip_route_category  category      
 17  Date                 object        
 18  Year                 int16         
 19  Month                i

In [12]:
# Call head to investigate our values and double check everything

df.head()

,bike_id,bike_type,duration,end_lat,end_lon,end_station,end_station_id,end_time,passholder_type,plan_duration,...,start_lon,start_station,start_station_id,start_time,trip_id,trip_route_category,Date,Year,Month,Day
0,11908,standard,30,39.93750,-75.1875,3063.0,NaN,2018-10-01 00:33:00,Indego30,30.0,...,-75.1875,3038.0,NaN,2018-10-01 00:03:00,299955254,One Way,2018-10-01,2018,10,0
1,05382,standard,7,39.93750,-75.1875,3064.0,NaN,2018-10-01 00:13:00,Indego30,30.0,...,-75.1875,3032.0,NaN,2018-10-01 00:06:00,299955253,One Way,2018-10-01,2018,10,0
2,11930,standard,6,39.93750,-75.1875,3064.0,NaN,2018-10-01 00:13:00,Indego30,30.0,...,-75.1875,3032.0,NaN,2018-10-01 00:07:00,299955252,One Way,2018-10-01,2018,10,0
3,05225,standard,6,39.93750,-75.1875,3007.0,NaN,2018-10-01 00:20:00,Indego30,30.0,...,-75.1875,3025.0,NaN,2018-10-01 00:14:00,299955251,One Way,2018-10-01,2018,10,0
4,03379,standard,6,39.96875,-75.1875,3035.0,NaN,2018-10-01 00:23:00,Indego30,30.0,...,-75.1875,3029.0,NaN,2018-10-01 00:17:00,299955250,One Way,2018-10-01,2018,10,0


In [13]:
# Set variables for Philadelphia's coordinates and create empty list to store forecast objects

latitude = 39.9509
longitude = -75.1575
forecasts = []

In [14]:
# Create empty dataframe to store in weather data read in from DarkSky forecast objects

Philly_Weather = pd.DataFrame(index = range(len(df['Date'].unique())), columns = ['Date', 'summary', 'temperature', 
                                                                      'precip_intensity', 'precip_probability','wind_speed',
                                                                      'humidity', 'mean_temperature', 'temperature6am',
                                                                      'temperatureNoon', 'temperature6pm', 'mean_precip_chance',
                                                                      'cumulative_rain', 'mean_wind_speed' , 'mean_humidity'])

In [15]:
# Create loop to communicate with Darksky API and read in Philadelphia forecasts for the dates we have Indego data for 

for i in range(len(Philly_Weather)):
    t = (df['Date'].unique()[i].isoformat())
    elements = t.split('-')
    elements = list(map(int , elements))
    t = dt(elements[0],elements[1],elements[2]).isoformat()
    forecasts.append(forecast(API_Key, latitude, longitude, time = t))

In [16]:
# Extract all weather info we want from forecast objects and read it into our weather dataframe 

for i in range(len(forecasts)):
    try:
        cumul_temperature = 0
        cumul_rain_chance = 0
        cumul_rain = 0
        cumul_wind = 0
        cumul_humidity = 0
        date = forecasts[i]
        Philly_Weather['Date'].iloc[i] = df['Date'].unique()[i]
        Philly_Weather['summary'].iloc[i] = forecasts[i].summary
        Philly_Weather['temperature'].iloc[i] = forecasts[i].temperature
        Philly_Weather['precip_intensity'].iloc[i] = forecasts[i].precipIntensity
        Philly_Weather['precip_probability'].iloc[i] = forecasts[i].precipProbability
        Philly_Weather['wind_speed'].iloc[i] = forecasts[i].windSpeed
        Philly_Weather['humidity'].iloc[i] = forecasts[i].humidity
        for j in range(24):
            try:
                cumul_rain_chance += date.hourly[j].precipProbability
                cumul_rain += date.hourly[j].precipIntensity
                cumul_temperature += date.hourly[j].temperature
                cumul_wind += date.hourly[j].windSpeed
                cumul_humidity += date.hourly[j].humidity
            except:
                pass
        Philly_Weather['mean_precip_chance'].iloc[i] = cumul_rain_chance / 24.0
        Philly_Weather['cumulative_rain'].iloc[i] = cumul_rain
        Philly_Weather['mean_temperature'].iloc[i] = cumul_temperature / 24.0
        Philly_Weather['mean_wind_speed'].iloc[i] = cumul_wind / 24.0
        Philly_Weather['mean_humidity'].iloc[i] = cumul_humidity / 24.0
        Philly_Weather['temperature6am'].iloc[i] = forecasts[i].hourly[5].temperature
        Philly_Weather['temperatureNoon'].iloc[i] = forecasts[i].hourly[11].temperature
        Philly_Weather['temperature6pm'].iloc[i] = forecasts[i].hourly[17].temperature
    except:
        pass

In [17]:
Philly_Weather['Date'] = pd.to_datetime(Philly_Weather['Date'], infer_datetime_format=True)

In [18]:
# Create new columns for year, month, and day of week to use for later analysis 

Philly_Weather['Year'] = Philly_Weather['Date'].dt.year
Philly_Weather['Month'] = Philly_Weather['Date'].dt.month
Philly_Weather['Day'] = Philly_Weather['Date'].dt.weekday

In [19]:
# Call head in order to check and make sure all of our weather data read in properly 

Philly_Weather.head()

,Date,summary,temperature,precip_intensity,precip_probability,wind_speed,humidity,mean_temperature,temperature6am,temperatureNoon,temperature6pm,mean_precip_chance,cumulative_rain,mean_wind_speed,mean_humidity,Year,Month,Day
0,2018-10-01,Partly Cloudy,63.2,0,0,1.59,0.81,70.4629,61.41,76.3,79.39,0.000833333,0.0023,4.00333,0.78125,2018,10,0
1,2018-10-02,Mostly Cloudy,68.41,0,0,2.32,0.92,70.3371,66.77,78.05,80.33,0.00333333,0.0038,4.96375,0.754583,2018,10,1
2,2018-10-03,Rain,65.33,0.0686,0.92,3.72,0.97,69.0888,64.79,71.29,77.14,0.085,0.317,3.35333,0.785,2018,10,2
3,2018-10-04,Clear,65.01,0,0,1.79,0.77,71.2529,62.8,72.78,82.43,0.0466667,0.1389,2.58333,0.485417,2018,10,3
4,2018-10-05,Overcast,73.57,0.0002,0.02,2.6,0.9,66.0721,64.93,63.89,67.52,0.00458333,0.0045,4.99333,0.693333,2018,10,4


In [20]:
# Call info to continue checking all of the information from our new weather data frame 

Philly_Weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                1712 non-null   datetime64[ns]
 1   summary             1710 non-null   object        
 2   temperature         1710 non-null   object        
 3   precip_intensity    1710 non-null   object        
 4   precip_probability  1710 non-null   object        
 5   wind_speed          1710 non-null   object        
 6   humidity            1710 non-null   object        
 7   mean_temperature    1710 non-null   object        
 8   temperature6am      1710 non-null   object        
 9   temperatureNoon     1710 non-null   object        
 10  temperature6pm      1710 non-null   object        
 11  mean_precip_chance  1710 non-null   object        
 12  cumulative_rain     1710 non-null   object        
 13  mean_wind_speed     1710 non-null   object      

In [21]:
# Remove rows where non date columns are null so that we have the same number of values in each column

Philly_Weather = Philly_Weather.dropna()

In [22]:
# Make sure the values all match up now 

Philly_Weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1710 entries, 0 to 1711
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                1710 non-null   datetime64[ns]
 1   summary             1710 non-null   object        
 2   temperature         1710 non-null   object        
 3   precip_intensity    1710 non-null   object        
 4   precip_probability  1710 non-null   object        
 5   wind_speed          1710 non-null   object        
 6   humidity            1710 non-null   object        
 7   mean_temperature    1710 non-null   object        
 8   temperature6am      1710 non-null   object        
 9   temperatureNoon     1710 non-null   object        
 10  temperature6pm      1710 non-null   object        
 11  mean_precip_chance  1710 non-null   object        
 12  cumulative_rain     1710 non-null   object        
 13  mean_wind_speed     1710 non-null   object      

In [23]:
# Create dummy variables for our forecast summary column so that it can later be used as an input for our model

Philly_Weather = pd.get_dummies(Philly_Weather, columns = ['summary'])

In [24]:
# One last check to make sure dummies loaded correctly and weather data is ready to be pickled

Philly_Weather.head()

,Date,temperature,precip_intensity,precip_probability,wind_speed,humidity,mean_temperature,temperature6am,temperatureNoon,temperature6pm,...,summary_Possible Drizzle and Humid,summary_Possible Flurries,summary_Possible Light Rain,summary_Possible Light Rain and Humid,summary_Possible Light Snow,summary_Rain,summary_Rain and Humid,summary_Rain and Windy,summary_Snow,summary_Windy
0,2018-10-01,63.2,0,0,1.59,0.81,70.4629,61.41,76.3,79.39,...,0,0,0,0,0,0,0,0,0,0
1,2018-10-02,68.41,0,0,2.32,0.92,70.3371,66.77,78.05,80.33,...,0,0,0,0,0,0,0,0,0,0
2,2018-10-03,65.33,0.0686,0.92,3.72,0.97,69.0888,64.79,71.29,77.14,...,0,0,0,0,0,1,0,0,0,0
3,2018-10-04,65.01,0,0,1.79,0.77,71.2529,62.8,72.78,82.43,...,0,0,0,0,0,0,0,0,0,0
4,2018-10-05,73.57,0.0002,0.02,2.6,0.9,66.0721,64.93,63.89,67.52,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Create a pickle file to store our weather data frame to store it in memory to continue being loaded for further use 

pickling_on = open("Weather.pickle","wb")
pickle.dump(Philly_Weather, pickling_on)
pickling_on.close()

In [26]:
# Create a pickle file to store our Indgo Bike data frame to store it in memory to continue being loaded for further use 

pickling_on_bike = open("Bikes.pickle","wb")
pickle.dump(df, pickling_on_bike)
pickling_on_bike.close()